In [5]:
from datetime import datetime
from time import sleep

In [21]:
init_time = datetime.now()
sleep(1.0)
now = datetime.now()
dt = now - init_time
dt_in_ms = dt.seconds *1000 + dt.microseconds / 1000

In [22]:
dt_in_ms

1000.522

In [23]:
dt

datetime.timedelta(seconds=1, microseconds=522)